# Data transformation and pipelines

Skforecast has two arguments in all the forecasters that allow more detailed control over input data transformations. This feature is particularly useful as many machine learning models require specific data pre-processing transformations. For example, linear models may benefit from features being scaled, or categorical features being encoded into numerical values.

+  `transformer_y`: an instance of a transformer (preprocessor) compatible with the scikit-learn preprocessing API with the methods: fit, transform, fit_transform and, inverse_transform. Scikit-learn ColumnTransformer is not allowed since they do not have the inverse_transform method. If multiple transformations are needed, a scikit-learn pipeline can be used. For example, scaling the target variable and applying a logarithmic transformation.

+ `transformer_exog`: an instance of a transformer (preprocessor) compatible with the scikit-learn preprocessing API. Scikit-learn ColumnTransformer can be used if the preprocessing transformations only apply to some specific columns or if different transformations are needed for different columns. For example, scale numeric features and one hot encode categorical ones. It is also possible to use a scikit-learn pipeline to apply multiple transformations sequentially.

Transformations are learned and applied before training the forecaster and are automatically used when calling `predict`. The output of `predict` is always on the same scale as the original series *y*.

## Libraries

In [28]:
# Libraries
# ==============================================================================
import numpy as np
import pandas as pd
from lightgbm import LGBMRegressor
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from skforecast.datasets import fetch_dataset
from skforecast.recursive import ForecasterRecursive
from skforecast.recursive import ForecasterRecursiveMultiSeries
from skforecast.model_selection import TimeSeriesFold
from skforecast.model_selection import grid_search_forecaster

## Data

In [29]:
# Download data
# ==============================================================================
data = fetch_dataset("h2o_exog")

h2o_exog
--------
Monthly expenditure ($AUD) on corticosteroid drugs that the Australian health
system had between 1991 and 2008. Two additional variables (exog_1, exog_2) are
simulated.
Hyndman R (2023). fpp3: Data for Forecasting: Principles and Practice (3rd
Edition). http://pkg.robjhyndman.com/fpp3package/,
https://github.com/robjhyndman/fpp3package, http://OTexts.com/fpp3.
Shape of the dataset: (195, 3)


In [30]:
# Data preprocessing
# ==============================================================================
data.index.name = 'date'
# Add an extra categorical variable
data['exog_3'] = (["A"] * int(len(data) / 2)) + (["B"] * (int(len(data) / 2) + 1))
data.head()

,y,exog_1,exog_2,exog_3
date,,,,
1992-04-01,0.379808,0.958792,1.166029,A
1992-05-01,0.361801,0.951993,1.117859,A
1992-06-01,0.410534,0.952955,1.067942,A
1992-07-01,0.483389,0.958078,1.097376,A
1992-08-01,0.475463,0.956370,1.122199,A


## Transforming input series

The following example shows how to include a transformer that scales the input series *y*.

In [31]:
# Create and fit forecaster that scales the input series
# ==============================================================================
forecaster = ForecasterRecursive(
                 regressor        = Ridge(random_state=123),
                 lags             = 3,
                 transformer_y    = StandardScaler(),
                 transformer_exog = None
             )

forecaster.fit(y=data['y'], exog=data[['exog_1', 'exog_2']])
forecaster

=================== 
ForecasterRecursive 
=================== 
Regressor: Ridge 
Lags: [1 2 3] 
Window features: None 
Window size: 3 
Exogenous included: True 
Exogenous names: exog_1, exog_2 
Transformer for y: StandardScaler() 
Transformer for exog: None 
Weight function included: False 
Differentiation order: None 
Training range: [Timestamp('1992-04-01 00:00:00'), Timestamp('2008-06-01 00:00:00')] 
Training index type: DatetimeIndex 
Training index frequency: MS 
Regressor parameters: 
    {'alpha': 1.0, 'copy_X': True, 'fit_intercept': True, 'max_iter': None,
    'positive': False, 'random_state': 123, 'solver': 'auto', 'tol': 0.0001} 
fit_kwargs: {} 
Creation date: 2024-12-09 11:29:14 
Last fit date: 2024-12-09 11:29:14 
Skforecast version: 0.14.0 
Python version: 3.12.4 
Forecaster id: None

## Transforming exogenous variables

The following example shows how to apply the same transformation (scaling) to all exogenous variables.

In [32]:
# Create and fit forecaster with same tranformation for all exogenous variables
# ==============================================================================
forecaster = ForecasterRecursive(
                 regressor        = Ridge(random_state=123),
                 lags             = 3,
                 transformer_y    = None,
                 transformer_exog = StandardScaler()
             )

forecaster.fit(y=data['y'], exog=data[['exog_1', 'exog_2']])
forecaster

=================== 
ForecasterRecursive 
=================== 
Regressor: Ridge 
Lags: [1 2 3] 
Window features: None 
Window size: 3 
Exogenous included: True 
Exogenous names: exog_1, exog_2 
Transformer for y: None 
Transformer for exog: StandardScaler() 
Weight function included: False 
Differentiation order: None 
Training range: [Timestamp('1992-04-01 00:00:00'), Timestamp('2008-06-01 00:00:00')] 
Training index type: DatetimeIndex 
Training index frequency: MS 
Regressor parameters: 
    {'alpha': 1.0, 'copy_X': True, 'fit_intercept': True, 'max_iter': None,
    'positive': False, 'random_state': 123, 'solver': 'auto', 'tol': 0.0001} 
fit_kwargs: {} 
Creation date: 2024-12-09 11:29:14 
Last fit date: 2024-12-09 11:29:14 
Skforecast version: 0.14.0 
Python version: 3.12.4 
Forecaster id: None

It is also possible to apply a different transformation to each exogenous variable making use of `ColumnTransformer`.

In [33]:
# Create and fit forecaster with different transformations for each exog variable
# ==============================================================================
transformer_exog = ColumnTransformer(
                       [('scale_1', StandardScaler(), ['exog_1']),
                        ('scale_2', StandardScaler(), ['exog_2']),
                        ('onehot', OneHotEncoder(), ['exog_3']),
                       ],
                       remainder = 'passthrough',
                       verbose_feature_names_out = False
                   )

forecaster = ForecasterRecursive(
                 regressor        = Ridge(random_state=123),
                 lags             = 3,
                 transformer_y    = None,
                 transformer_exog = transformer_exog
             )

forecaster.fit(y=data['y'], exog=data[['exog_1', 'exog_2', 'exog_3']])
forecaster

=================== 
ForecasterRecursive 
=================== 
Regressor: Ridge 
Lags: [1 2 3] 
Window features: None 
Window size: 3 
Exogenous included: True 
Exogenous names: exog_1, exog_2, exog_3 
Transformer for y: None 
Transformer for exog: ColumnTransformer(remainder='passthrough',
                  transformers=[('scale_1', StandardScaler(), ['exog_1']),
                                ('scale_2', StandardScaler(), ['exog_2']),
                                ('onehot', OneHotEncoder(), ['exog_3'])],
                  verbose_feature_names_out=False) 
Weight function included: False 
Differentiation order: None 
Training range: [Timestamp('1992-04-01 00:00:00'), Timestamp('2008-06-01 00:00:00')] 
Training index type: DatetimeIndex 
Training index frequency: MS 
Regressor parameters: 
    {'alpha': 1.0, 'copy_X': True, 'fit_intercept': True, 'max_iter': None,
    'positive': False, 'random_state': 123, 'solver': 'auto', 'tol': 0.0001} 
fit_kwargs: {} 
Creation date: 2024-12-09 11:29:14 
Last fit date: 2024-12-09 11:29:14 
Skforecast version: 0.14.0 
Python version: 3.12.4 
Forecaster id: None

It is possible to verify if the data transformation has been applied correctly by examining the training matrices. The training matrices should reflect the data transformation that was specified using the `transformer_y` or `transformer_exog` arguments.

In [34]:
X_train, y_train = forecaster.create_train_X_y(
                       y    = data['y'],
                       exog = data[['exog_1', 'exog_2', 'exog_3']]
                   )

In [35]:
X_train.head(4)

,lag_1,lag_2,lag_3,exog_1,exog_2,exog_3_A,exog_3_B
date,,,,,,,
1992-07-01,0.410534,0.361801,0.379808,-2.119529,-2.135088,1.0,0.0
1992-08-01,0.483389,0.410534,0.361801,-2.131024,-1.996017,1.0,0.0
1992-09-01,0.475463,0.483389,0.410534,-2.109222,-1.822392,1.0,0.0
1992-10-01,0.534761,0.475463,0.483389,-2.132137,-1.590667,1.0,0.0


In [36]:
y_train.head(4)

date
1992-07-01    0.483389
1992-08-01    0.475463
1992-09-01    0.534761
1992-10-01    0.568606
Freq: MS, Name: y, dtype: float64

<div class="admonition note" name="html-admonition" style="background: rgba(0,191,191,.1); padding-top: 0px; padding-bottom: 6px; border-radius: 8px; border-left: 8px solid #00bfa5; border-color: #00bfa5; padding-left: 10px; padding-right: 10px;">

<p class="title">
    <i style="font-size: 18px; color:#00bfa5;"></i>
    <b style="color: #00bfa5;">&#128161 Tip</b>
</p>

To learn more about how to extract the training and prediction matrices, visit the following link: <a href="../user_guides/training-and-prediction-matrices.html">How to Extract Training and Prediction Matrices</a>.

</div>

## Sklearn Pipelines and ColumnTransformer

Both `transformer_y` and `transformer_exog` accept scikit-learn pipelines. This allows for multiple transformations to be applied sequentially. The following example shows how to scale the target variable and apply a box-cox transformation.

In [37]:
# Multiple transformations using pipelines and column transformers
# ==============================================================================
transformer_y = Pipeline(
                    steps = [
                        ('scaler', StandardScaler()),
                        ('power', PowerTransformer(method='yeo-johnson'))
                    ]
                )

transformer_exog = ColumnTransformer(
                       [('scale_1', StandardScaler(), ['exog_1']),
                        ('scale_2', StandardScaler(), ['exog_2']),
                        ('onehot', OneHotEncoder(), ['exog_3']),
                       ],
                       remainder = 'passthrough',
                       verbose_feature_names_out = False
                   )

forecaster = ForecasterRecursive(
                    regressor        = Ridge(random_state=123),
                    lags             = 3,
                    transformer_y    = transformer_y,
                    transformer_exog = transformer_exog
                )
forecaster.fit(y=data['y'], exog=data[['exog_1', 'exog_2', 'exog_3']])
forecaster

=================== 
ForecasterRecursive 
=================== 
Regressor: Ridge 
Lags: [1 2 3] 
Window features: None 
Window size: 3 
Exogenous included: True 
Exogenous names: exog_1, exog_2, exog_3 
Transformer for y: Pipeline(steps=[('scaler', StandardScaler()), ('power', PowerTransformer())]) 
Transformer for exog: ColumnTransformer(remainder='passthrough',
                  transformers=[('scale_1', StandardScaler(), ['exog_1']),
                                ('scale_2', StandardScaler(), ['exog_2']),
                                ('onehot', OneHotEncoder(), ['exog_3'])],
                  verbose_feature_names_out=False) 
Weight function included: False 
Differentiation order: None 
Training range: [Timestamp('1992-04-01 00:00:00'), Timestamp('2008-06-01 00:00:00')] 
Training index type: DatetimeIndex 
Training index frequency: MS 
Regressor parameters: 
    {'alpha': 1.0, 'copy_X': True, 'fit_intercept': True, 'max_iter': None,
    'positive': False, 'random_state': 123, 'solver': 'auto', 'tol': 0.0001} 
fit_kwargs: {} 
Creation date: 2024-12-09 11:29:14 
Last fit date: 2024-12-09 11:29:14 
Skforecast version: 0.14.0 
Python version: 3.12.4 
Forecaster id: None

## Custom transformers

Using scikit-learn [FunctionTransformer](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.FunctionTransformer.html) it is possible to include custom transformers in the forecaster object, for example, a logarithmic transformation. 


Scikit-learn's [FunctionTransformer](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.FunctionTransformer.html) can be used to incorporate custom transformers, such as a logarithmic transformation, in the forecaster object. To implement this, a user-defined transformation function can be created and then passed to the `FunctionTransformer`. Detailed information on how to use FunctionTransformer can be found in the [scikit-learn documentation](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.FunctionTransformer.html).

<div class="admonition note" name="html-admonition" style="background: rgba(255,145,0,.1); padding-top: 0px; padding-bottom: 6px; border-radius: 8px; border-left: 8px solid #ff9100; border-color: #ff9100; padding-left: 10px; padding-right: 10px">

<p class="title">
    <i style="font-size: 18px; color:#ff9100; border-color: #ff1744;"></i>
    <b style="color: #ff9100;"> <span style="color: #ff9100;">&#9888;</span> Warning</b>
</p>

For versions 1.1.0 >= scikit-learn <= 1.2.0 <code>sklearn.preprocessing.FunctionTransformer.inverse_transform</code> does not support DataFrames that are all numerical when <code>check_inverse=True</code>. It will raise an Exception which is fixed in scikit-learn 1.2.1.

More info: https://scikit-learn.org/stable/whats_new/v1.2.html#version-1-2-1

</div>

In [38]:
# Create custom transformer
# =============================================================================
def log_transform(x):
    """ 
    Calculate log adding 1 to avoid calculation errors if x is very close to 0.
    """
    return np.log(x + 1)


def exp_transform(x):
    """
    Inverse of log_transform.
    """
    return np.exp(x) - 1


transformer_y = FunctionTransformer(func=log_transform, inverse_func=exp_transform)

# Create and train forecaster
# ==============================================================================
forecaster = ForecasterRecursive(
                 regressor     = Ridge(random_state=123),
                 lags          = 3,
                 transformer_y = transformer_y
             )

forecaster.fit(y=data['y'])

If the `FunctionTransformer` has an inverse function, the output of the predict method is automatically transformed back to the original scale.

In [39]:
forecaster.predict(steps=4)

2008-07-01    0.776206
2008-08-01    0.775471
2008-09-01    0.777200
2008-10-01    0.777853
Freq: MS, Name: pred, dtype: float64

## Pipelines as regressors

<div class="admonition note" name="html-admonition" style="background: rgba(255,145,0,.1); padding-top: 0px; padding-bottom: 6px; border-radius: 8px; border-left: 8px solid #ff9100; border-color: #ff9100; padding-left: 10px; padding-right: 10px">

<p class="title">
    <i style="font-size: 18px; color:#ff9100; border-color: #ff1744;"></i>
    <b style="color: #ff9100;"> <span style="color: #ff9100;">&#9888;</span> Warning</b>
</p>

Skforecast allows the usage of scikit-learn pipelines as regressors. It is important to note that ColumnTransformer cannot be included in the pipeline; thus, the same transformation will be applied to both the modeled series and all exogenous variables. However, if the preprocessing transformations only apply to specific columns, then they need to be applied separately using <code>transformer_y</code> and <code>transformer_exog</code>.

</div>

In [40]:
pipe = Pipeline(steps=[('scaler', StandardScaler()), ('regressor', Ridge())])
pipe

Pipeline(steps=[('scaler', StandardScaler()), ('regressor', Ridge())])

In [41]:
# Create and fit forecaster
# ==============================================================================
forecaster = ForecasterRecursive(
                 regressor = pipe,
                 lags      = 10
             )

forecaster.fit(y=data['y'], exog=data[['exog_1', 'exog_2']])
forecaster

=================== 
ForecasterRecursive 
=================== 
Regressor: Pipeline 
Lags: [ 1  2  3  4  5  6  7  8  9 10] 
Window features: None 
Window size: 10 
Exogenous included: True 
Exogenous names: exog_1, exog_2 
Transformer for y: None 
Transformer for exog: None 
Weight function included: False 
Differentiation order: None 
Training range: [Timestamp('1992-04-01 00:00:00'), Timestamp('2008-06-01 00:00:00')] 
Training index type: DatetimeIndex 
Training index frequency: MS 
Regressor parameters: 
    {'scaler__copy': True, 'scaler__with_mean': True, 'scaler__with_std': True,
    'regressor__alpha': 1.0, 'regressor__copy_X': True,
    'regressor__fit_intercept': True, 'regressor__max_iter': None,
    'regressor__positive': False, 'regressor__random_state': None,
    'regressor__solver': 'auto', 'regressor__tol': 0.0001} 
fit_kwargs: {} 
Creation date: 2024-12-09 11:29:15 
Last fit date: 2024-12-09 11:29:15 
Skforecast version: 0.14.0 
Python version: 3.12.4 
Forecaster id: None

When performing a grid search over a scikit-learn pipeline, the model's name precedes the parameters' name.

In [42]:
# Hyperparameter grid search using a scikit-learn pipeline
# ==============================================================================
pipe = Pipeline(steps=[('scaler', StandardScaler()), ('regressor', Ridge())])
forecaster = ForecasterRecursive(
                 regressor = pipe,
                 lags = 10  # This value will be replaced in the grid search
             )

# Regressor's hyperparameters
param_grid = {'regressor__alpha': np.logspace(-3, 5, 10)}

# Lags used as predictors
lags_grid = [5, 24, [1, 2, 3, 23, 24]]

cv = TimeSeriesFold(
    steps=5, initial_train_size=len(data.loc[:'2000-04-01']), refit=False
)

results_grid = grid_search_forecaster(
                   forecaster         = forecaster,
                   y                  = data['y'],
                   exog               = data[['exog_1', 'exog_2']],
                   param_grid         = param_grid,
                   lags_grid          = lags_grid,
                   cv                 = cv,
                   metric             = 'mean_absolute_error',
                   return_best        = True,
                   verbose            = False,
                   show_progress      = True
               )

results_grid.head(4)

lags grid:   0%|          | 0/3 [00:00<?, ?it/s]

params grid:   0%|          | 0/10 [00:00<?, ?it/s]

`Forecaster` refitted using the best-found lags and parameters, and the whole data set: 
  Lags: [1 2 3 4 5] 
  Parameters: {'regressor__alpha': 0.001}
  Backtesting metric: 6.845311709556121e-05


,lags,lags_label,params,mean_absolute_error,regressor__alpha
0,"[1, 2, 3, 4, 5]","[1, 2, 3, 4, 5]",{'regressor__alpha': 0.001},0.000068,0.001000
1,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",{'regressor__alpha': 0.001},0.000188,0.001000
2,"[1, 2, 3, 4, 5]","[1, 2, 3, 4, 5]",{'regressor__alpha': 0.007742636826811269},0.000526,0.007743
3,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",{'regressor__alpha': 0.007742636826811269},0.001413,0.007743


## Transforming multiple input series in global models

When using global forecasting models (<a href="../user_guides/independent-multi-time-series-forecasting.html#scikit-learn-transformers-in-multi-series">ForecasterRecursiveMultiSeries</a> or <a href="../user_guides/dependent-multi-series-multivariate-forecasting.html#scikit-learn-transformers-in-multivariate">ForecasterDirectMultiVariate</a>) the <code>transformer_series</code> argument replaces <code>transformer_y</code>. Three different options are available:

+ `transformer_series` is a single transformer: When a single transformer is provided, it is automatically cloned for each individual series. Each cloned transformer is then trained separately on one of the series.

+ `transformer_series` is a dictionary: A different transformer can be specified for each series by passing a dictionary where the keys correspond to the series names and the values are the transformers. Each series is transformed according to its designated transformer. When this option is used, it is mandatory to include a transformer for unknown series, which is indicated by the key `'_unknown_level'`.

+ `transformer_series` is None: no transformations are applied to any of the series.

Regardless of the configuration, each series is transformed independently. Even when using a single transformer, it is cloned internally and applied separately to each series.

In [43]:
# Data download
# ==============================================================================
data = fetch_dataset(name="items_sales")
data.head()

items_sales
-----------
Simulated time series for the sales of 3 different items.
Simulated data.
Shape of the dataset: (1097, 3)


,item_1,item_2,item_3
date,,,
2012-01-01,8.253175,21.047727,19.429739
2012-01-02,22.777826,26.578125,28.009863
2012-01-03,27.549099,31.751042,32.078922
2012-01-04,25.895533,24.567708,27.252276
2012-01-05,21.379238,18.191667,20.357737


In [44]:
# Series transformation: same transformation for all series
# ==============================================================================
forecaster = ForecasterRecursiveMultiSeries(
                 regressor          = LGBMRegressor(random_state=123, verbose=-1),
                 lags               = 24,
                 encoding           = 'ordinal',
                 transformer_series = StandardScaler(),
                 transformer_exog   = None
             )
forecaster.fit(series=data)

It is possible to access the fitted transformers for each series through the `transformers_series_` attribute. This allows verification that each transformer has been trained independently.


In [45]:
# Mean and scale of the transformer for each series
# ==============================================================================
for k, v in forecaster.transformer_series_.items():
    print(f"Series {k}: {v} mean={v.mean_}, scale={v.scale_}")

Series item_1: StandardScaler() mean=[22.37366364], scale=[2.54258317]
Series item_2: StandardScaler() mean=[16.26942518], scale=[4.89965692]
Series item_3: StandardScaler() mean=[17.19276546], scale=[5.43694388]
Series _unknown_level: StandardScaler() mean=[18.61195143], scale=[5.21803675]


In [46]:
# Series transformation: different transformation for each series
# ==============================================================================
forecaster = ForecasterRecursiveMultiSeries(
                 regressor          = LGBMRegressor(random_state=123, verbose=-1),
                 lags               = 24,
                 encoding           = 'ordinal',
                 transformer_series = {'item_1': StandardScaler(), 'item_2': MinMaxScaler(), '_unknown_level': StandardScaler()},
                 transformer_exog   = None
             )

forecaster.fit(series=data)

c:\anaconda\envs\skforecast_14_p12\Lib\site-packages\skforecast\utils\utils.py:363: IgnoredArgumentWarning: {'item_3'} not present in `transformer_series`. No transformation is applied to these series. 
 You can suppress this warning using: warnings.simplefilter('ignore', category=IgnoredArgumentWarning)
  warnings.warn(


In [47]:
# Transformer trained for each series
# ==============================================================================
for k, v in forecaster.transformer_series_.items():
    if v is not None:
        print(f"Series {k}: {v.get_params()}")
    else:
        print(f"Series {k}: {v}")

Series item_1: {'copy': True, 'with_mean': True, 'with_std': True}
Series item_2: {'clip': False, 'copy': True, 'feature_range': (0, 1)}
Series item_3: None
Series _unknown_level: {'copy': True, 'with_mean': True, 'with_std': True}
